In [1]:
from google.colab import drive
drive.mount("/content/drive")

!pip install torchtext -qq
!pip install transformers evaluate wandb datasets accelerate -U -qq

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.6 MB/s eta 0:

In [2]:
import torch
import torch.nn as nn
from torchtext.vocab import vocab
import random
import numpy as np
import joblib
from collections import Counter
from pathlib import Path
from sklearn.metrics import confusion_matrix
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
import base64
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TrainingArguments, Trainer
from functools import partial
import sys

In [3]:
basepath = '/content/drive/MyDrive/Colab_Notebooks/BUAN_6342_Applied_Natural_Language_Processing'
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/BUAN_6342_Applied_Natural_Language_Processing/0_Custom_files')
# Set the base folder path using the Path class for better path handling
base_folder = Path(basepath)

# Define the data folder path by appending the relative path to the base folder
# This is where the data files will be stored
data_folder = base_folder / '0_Data_Folder'

# Define the model folder path for saving trained models
# This path points to a specific folder designated for NLP models related to the IMDb dataset
model_folder = data_folder

custom_functions = base_folder / '0_Custom_files'
# Create the model folder directory. If it already exists, do nothing.
# The 'parents=True' argument ensures that all parent directories are created if they don't exist.
model_folder.mkdir(exist_ok=True, parents=True)

# Create the data folder directory in a similar manner.
data_folder.mkdir(exist_ok=True, parents=True)

In [12]:
if 'google.colab' in str(get_ipython()):
    !chmod 600 /content/drive/MyDrive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Data/.kaggle/kaggle.json
    !ls -la /content/drive/MyDrive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Data/.kaggle
else:
    !chmod 600 '/Users/harikrishnadev/Library/CloudStorage/GoogleDrive-harikrish0607@gmail.com/My Drive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Data/.kaggle/kaggle.json'
    ! ls -la '/Users/harikrishnadev/Library/CloudStorage/GoogleDrive-harikrish0607@gmail.com/My Drive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Data/.kaggle'

total 1
-rw------- 1 root root 70 Nov 27 02:27 kaggle.json


In [ ]:
import os
if 'google.colab' in str(get_ipython()):
    os.environ['KAGGLE_CONFIG_DIR']='/content/drive/MyDrive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Data/.kaggle'
else:
    os.environ['KAGGLE_CONFIG_DIR']='/Users/harikrishnadev/Library/CloudStorage/GoogleDrive-harikrish0607@gmail.com/My Drive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Data/.kaggle'

In [15]:
! kaggle competitions download -c emotion-detection-spring2014

100% 609k/609k [00:00<00:00, 1.42MB/s]
100% 609k/609k [00:00<00:00, 1.42MB/s]


In [16]:
! unzip emotion-detection-spring2014.zip

Archive:  emotion-detection-spring2014.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [18]:
df_train = pd.read_csv('train.csv', usecols=lambda column: column != 'ID')

In [19]:
df_test = pd.read_csv('test.csv', usecols=lambda column: column != 'ID')

In [20]:
df_train.columns

Index(['Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
       'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
      dtype='object')

In [21]:
df_test.columns

Index(['Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
       'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
      dtype='object')

In [22]:

X_train_text = df_train["Tweet"]

X_test_text = df_test["Tweet"]

In [23]:
X_train_text = X_train_text.to_list()

X_test = X_test_text.to_list()

In [24]:
y_train_labels = df_train.drop("Tweet", axis=1)

y_test = df_test.drop("Tweet", axis=1)

y_train_labels = y_train_labels.to_numpy()

y_test = y_test.to_numpy()

In [25]:

X_train, X_valid, y_train, y_valid = train_test_split(X_train_text,y_train_labels, test_size=0.3,random_state= 23 , shuffle = True)


In [26]:
trainset = Dataset.from_dict({
    'texts': X_train,
    'labels': y_train
})

validset = Dataset.from_dict({
    'texts': X_valid,
    'labels': y_valid
})

testset = Dataset.from_dict({
    'texts': X_test,
    'labels': y_test
})


In [27]:
trainset.features

{'texts': Value(dtype='string', id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [28]:
trainset[1:5]

{'texts': ['Be happy not because everything is good, but because you can see the good side of everything ',
  'Watch this amazing live.ly broadcast by @katyy.mae #lively #musically',
  "'We can throw stones, complain about them, stumble on them, climb over them, or build with them.' — William Arthur Ward. #inspiring",
  'Huns are like a box of coffee revels #horrible'],
 'labels': [[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
  [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]}

In [29]:
class CustomConfiguration(PretrainedConfig):
  def __init__(self, vocab_size=0, embedding_dim=0, hidden_dim1=0, hidden_dim2=0, num_labels=11, **kwargs):
      super().__init__()
      self.vocab_size = vocab_size
      self.embedding_dim = embedding_dim
      self.hidden_dim1 = hidden_dim1
      self.hidden_dim2 = hidden_dim2
      self.num_labels = num_labels

In [30]:
class CustomMLP(PreTrainedModel):
  config_class = CustomConfiguration
  def __init__(self, config):
    super().__init__(config)

    self.embedding_bag = nn.EmbeddingBag(config.vocab_size, config.embedding_dim)
    self.layers = nn.Sequential(
            nn.Linear(config.embedding_dim, config.hidden_dim1),
            nn.BatchNorm1d(num_features=config.hidden_dim1),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(config.hidden_dim1, config.hidden_dim2),
            nn.BatchNorm1d(num_features=config.hidden_dim2),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(config.hidden_dim2, config.num_labels)
        )

  def forward(self,input_ids,offsets, labels=None):
    embed_out = self.embedding_bag(input_ids, offsets)
    logits = self.layers(embed_out)
    loss = None
    if labels is not None:
      loss_fct = nn.BCEWithLogitsLoss()
      loss = loss_fct(logits, labels)
    return SequenceClassifierOutput(loss=loss,logits=logits)

In [31]:
def get_vocab(dataset, min_freq= 1):
  counter = Counter()
  for text in dataset["texts"]:
    counter.update(str(text).split())
  my_vocab = vocab(counter,min_freq = min_freq)
  my_vocab.insert_token('<unk>',0)
  my_vocab.set_default_index(0)
  return my_vocab

In [32]:
def tokenizer(text,vocab):
  return [vocab[token] for token in str(text).split()]

In [33]:
def collate_batch(batch,my_vocab):
  texts = [sample["texts"] for sample in batch]
  labels = [sample["labels"] for sample in batch]
  labels = torch.tensor(labels, dtype=torch.float32)
  list_of_list_of_indices = [tokenizer(text,my_vocab) for text in texts]
  input_ids = torch.cat([torch.tensor(i,dtype=torch.int64)for i in list_of_list_of_indices])
  offsets = [0] + [len(i) for i in list_of_list_of_indices]
  offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
  return {
        'input_ids': input_ids,
        'offsets': offsets,
        'labels': labels
    }

In [34]:
emoticons_vocab = get_vocab(trainset,min_freq=2)
collate_fn = partial(collate_batch,my_vocab = emoticons_vocab)

In [35]:
config = CustomConfiguration(vocab_size=len(emoticons_vocab),
                         embedding_dim=300,
                         hidden_dim1=200,
                         hidden_dim2=100,
                         num_labels=11)

In [36]:
config.id2label = {0: 'anger', 1: 'anticipation', 2: 'disgust', 3: 'fear', 4: 'joy' , 5: 'love', 6: 'optimism', 7: 'pessimism', 8: 'sadness', 9: 'surprise', 10: 'trust'}

config.label2id = {v:k for k,v  in config.id2label.items()}

config

CustomConfiguration {
  "embedding_dim": 300,
  "hidden_dim1": 200,
  "hidden_dim2": 100,
  "id2label": {
    "0": "anger",
    "1": "anticipation",
    "2": "disgust",
    "3": "fear",
    "4": "joy",
    "5": "love",
    "6": "optimism",
    "7": "pessimism",
    "8": "sadness",
    "9": "surprise",
    "10": "trust"
  },
  "label2id": {
    "anger": 0,
    "anticipation": 1,
    "disgust": 2,
    "fear": 3,
    "joy": 4,
    "love": 5,
    "optimism": 6,
    "pessimism": 7,
    "sadness": 8,
    "surprise": 9,
    "trust": 10
  },
  "transformers_version": "4.39.3",
  "vocab_size": 7088
}

In [38]:
model = CustomMLP(config = config)

In [68]:
import evaluate

def compute_metrics(eval_pred):
  combined_metrics = evaluate.combine([evaluate.load("accuracy"), evaluate.load("f1", average = "macro")])
  logits, labels = eval_pred
  predictions = (logits>0.5).astype(int).reshape(-1)
  evaluations = combined_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))
  return evaluations

In [69]:
training_args = TrainingArguments(
    num_train_epochs=5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    weight_decay=0.1,
    learning_rate=0.001,
    optim='adamw_torch',
    remove_unused_columns=False,
    output_dir=str(model_folder),
    evaluation_strategy='steps',
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_strategy='steps',
    logging_steps=50,
    report_to='wandb',
    run_name='tweet_hf_trainer',
)

In [70]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset = validset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [71]:
import wandb
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    wandb.login(key=userdata.get('wandb'))
else:
    !wandb login

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [72]:
wandb.init(project="nlp_course_spring_2024-emotion-analysis-hf-trainer-rnn")

eval/f1_0_f1,▁
eval/f1_1_f1,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁
train/global_step,▁▁
train/grad_norm,▁
train/learning_rate,▁
train/loss,▁


In [73]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
50,0.406500,0.461300,0.791317,0.080207
100,0.450700,0.455513,0.794494,0.116358
150,0.435600,0.454432,0.797121,0.174302
200,0.423700,0.449456,0.798298,0.178566


TrainOutput(global_step=215, training_loss=0.42934231869010037, metrics={'train_runtime': 33.2463, 'train_samples_per_second': 813.023, 'train_steps_per_second': 6.467, 'total_flos': 5726387638890.0, 'train_loss': 0.42934231869010037, 'epoch': 5.0})

In [74]:
trainer.evaluate()

{'eval_loss': 0.44945552945137024,
 'eval_accuracy': 0.7982979057180956,
 'eval_f1': 0.17856572432518766,
 'eval_runtime': 13.544,
 'eval_samples_per_second': 171.146,
 'eval_steps_per_second': 1.403,
 'epoch': 5.0}

In [75]:
valid_output = trainer.predict(validset)

In [76]:
valid_output._fields

('predictions', 'label_ids', 'metrics')

In [77]:
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")

The best model was saved at step 200.


In [78]:
valid_output

PredictionOutput(predictions=array([[-0.53479517, -0.889774  , -0.44687757, ..., -0.6316327 ,
        -1.5251265 , -1.9906536 ],
       [-1.7839837 , -1.3153118 , -1.5540483 , ..., -1.116769  ,
        -2.5738368 , -1.7146027 ],
       [-0.16810036, -1.4289472 ,  0.05627719, ...,  0.04064272,
        -2.5803697 , -2.5987508 ],
       ...,
       [-1.2568899 , -1.5243597 , -1.2398703 , ..., -0.59277076,
        -2.5317035 , -2.4199162 ],
       [ 0.09387343, -1.0712675 ,  0.03638291, ..., -0.33844033,
        -2.2442095 , -2.0251577 ],
       [-3.1750836 , -1.0311128 , -2.8185358 , ..., -1.5713576 ,
        -3.1530511 , -2.2485118 ]], dtype=float32), label_ids=array([[1., 1., 1., ..., 0., 1., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 1., 0., 0.]], dtype=float32), metrics={'test_loss': 0.44945552945137024, 'test_accuracy': 0.7982979057180

In [79]:
list_of_list_of_indices = [tokenizer(text, emoticons_vocab) for text in testset]
offsets = [0] + [len(i) for i in list_of_list_of_indices]
offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
indices = torch.cat([torch.tensor(i, dtype=torch.int64) for i in list_of_list_of_indices])

In [81]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
indices = indices.to(device)
offsets = offsets.to(device)
outputs = model(indices,offsets)

In [82]:
outputs = model(indices,offsets)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.3596, -1.0133, -0.2939,  ..., -1.2402, -2.8085, -2.7141],
        [-0.6814, -1.3986, -0.5276,  ..., -1.3516, -2.8909, -2.9462],
        [-0.6682, -1.4048, -0.7492,  ..., -1.3264, -2.8297, -2.8878],
        ...,
        [-0.3446, -1.1781, -0.3249,  ..., -1.2323, -2.7971, -2.5167],
        [-0.5814, -1.3962, -0.4446,  ..., -1.1301, -2.7861, -2.9331],
        [-0.4571, -1.2334, -0.5097,  ..., -1.4597, -2.4433, -2.7385]],
       device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)